In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Concatenate
from keras import optimizers
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from keras import backend as K
from tqdm import tqdm


# reserve only the 30% of the GPU memory
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

/data/user/adipilat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 512
width = 50
height = 10
channels = 4 #3
classes = 4
epochs = 100
dataset_dir = '/data/user/adipilat/ParticleID/genEvts/new_datasets/padded/'
save_dir = '/data/user/adipilat/ParticleID/models/'
padding = 'padding' + str(height)
model_name= padding +'_PCAModel'
history_name = model_name + '_history'

# This dictionary should be extended to new classes and antiparticles
class_labels = {22:0, 11:1, 13:2, 211:3}

In [3]:
# arrays of data needed for training

data_array = []
pid_array = []
en_array = []

# read dataset
files = [f for f in os.listdir(dataset_dir) if f.endswith("h5")]

for name in tqdm(files):
    print("Reading file", name)
    data = pd.read_hdf(dataset_dir + name)
    num_tracks = int(0.9 * data.trackster.max()) #with the new dataset, we'll split into training and test
    for i in range(1, num_tracks+1):
        track = data[data['trackster'] == i]
#         img = np.array([track.E.values, track.eta.values, track.phi.values]).T.reshape(width, height, channels)
#         img = np.array([track.E.values, np.abs(track.eta.values-track.eta_mean.values), np.abs(track.phi.values-track.phi_mean.values)]).T.reshape(width, height, channels)
        img = np.array([track.E.values, track.x_pca.values, track.y_pca.values, track.z_pca.values]).T.reshape(width, height, channels)
        pid_vals, pid_counts = np.unique(track[track['cpID'] != 0].cpID, return_counts=True)
        pid_index = np.argmax(pid_counts)
        pid = int(pid_vals[pid_index])
        pid = class_labels[pid]
        en_value = track[track['genE'] != 0].genE.max()
        data_array.append(img)
        pid_array.append(pid)
        en_array.append(en_value)

data_array = np.array(data_array)
pid_array = np.array(pid_array)
pid_array = keras.utils.to_categorical(pid_array, num_classes=classes, dtype='float32')
en_array = np.array(en_array)

  0%|          | 0/4 [00:00<?, ?it/s]

Reading file pion_c_newPadded.h5


 25%|██▌       | 1/4 [00:51<02:33, 51.01s/it]

Reading file gamma_newPadded.h5


 50%|█████     | 2/4 [01:41<01:42, 51.00s/it]

Reading file electron_newPadded.h5


 75%|███████▌  | 3/4 [02:32<00:50, 50.96s/it]

Reading file muon_newPadded.h5


100%|██████████| 4/4 [03:23<00:00, 50.90s/it]


In [4]:
print(data_array.shape)
print(pid_array.shape)
print(en_array.shape)

(36000, 50, 10, 4)
(36000, 4)
(36000,)


In [5]:
mean_en = np.mean(en_array)
std_en = np.std(en_array)
print('Mean Energy Value: {}'.format(mean_en))
print('Std Energy Value: {}'.format(std_en))

en_array_norm = (en_array - mean_en)/std_en

Mean Energy Value: 213.90352475881576
Std Energy Value: 108.05413626100672


In [6]:
print('Creating model...')

def full_model():

    input_img = Input(shape=(width, height, channels), name='input')
    
    conv = Conv2D(3, (5,1), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv1')(input_img)
    conv = Conv2D(3, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv2')(conv)
    conv = Conv2D(3, (3,3), activation='relu', padding='same', kernel_initializer='random_uniform', data_format='channels_last', name='conv3')(conv)

    flat = Flatten()(conv)

    dense = Dense(512, activation='relu', kernel_initializer='random_uniform', name='dense1')(flat)
#     drop = Dropout(0.4)(dense)
    dense = Dense(128, activation='relu', kernel_initializer='random_uniform', name='dense2')(dense)
#     drop = Dropout(0.5)(dense)

    dense_id = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_id1')(dense)
#     drop_id = Dropout(0.5)(dense_id)
    dense_id = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_id2')(dense_id)
#     drop_id = Dropout(0.5)(dense_id)

    pid = Dense(classes, activation='softmax', kernel_initializer='random_uniform', name='pid_output')(dense_id)

    dense_er = Dense(64, activation='relu', kernel_initializer='random_uniform', name='dense_er1')(dense)
    dense_er = Dense(16, activation='relu', kernel_initializer='random_uniform', name='dense_er2')(dense_er)
    
    enreg = Dense(1, name='enreg_output')(dense_er)

    model = Model(inputs=input_img, outputs=[pid, enreg])

    model.compile(loss={'pid_output': 'categorical_crossentropy', 'enreg_output': 'mse'}, loss_weights={'pid_output': 1, 'enreg_output': 1}, optimizer='adam', metrics={'pid_output': 'accuracy', 'enreg_output': 'mse'})
    return model

model = full_model()
model.summary()

print("Model created!")

Creating model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 50, 10, 4)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 50, 10, 3)    63          input[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 50, 10, 3)    84          conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 50, 10, 3)    84          conv2[0][0]                      
___________________________________________________________________________________________

In [7]:
history = model.fit(data_array, {'pid_output': pid_array, 'enreg_output': en_array_norm}, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)], shuffle=True, verbose=1)
history_save = pd.DataFrame(history.history).to_hdf(save_dir + history_name + ".h5", "history", append=False)

Train on 32400 samples, validate on 3600 samples
Epoch 1/100
32400/32400 [==============================] - 2s 69us/step - loss: 1.9079 - pid_output_loss: 1.2499 - enreg_output_loss: 0.6580 - pid_output_acc: 0.3014 - enreg_output_mean_squared_error: 0.6580 - val_loss: 2.6166 - val_pid_output_loss: 1.4591 - val_enreg_output_loss: 1.1576 - val_pid_output_acc: 0.0000e+00 - val_enreg_output_mean_squared_error: 1.1576
Epoch 2/100
32400/32400 [==============================] - 1s 20us/step - loss: 1.0241 - pid_output_loss: 0.7435 - enreg_output_loss: 0.2805 - pid_output_acc: 0.5992 - enreg_output_mean_squared_error: 0.2805 - val_loss: 2.3751 - val_pid_output_loss: 1.3485 - val_enreg_output_loss: 1.0266 - val_pid_output_acc: 0.9644 - val_enreg_output_mean_squared_error: 1.0266
Epoch 3/100
32400/32400 [==============================] - 1s 20us/step - loss: 0.7599 - pid_output_loss: 0.5208 - enreg_output_loss: 0.2390 - pid_output_acc: 0.7321 - enreg_output_mean_squared_error: 0.2390 - val_loss:

In [8]:
# Save model and weights
model.save(save_dir + model_name + ".h5")
print('Saved trained model at %s ' % save_dir)

Saved trained model at /data/user/adipilat/ParticleID/models/ 


In [9]:
# save the frozen model
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pbtxt", as_text=True)
tf.train.write_graph(frozen_graph, save_dir, model_name + ".pb", as_text=False)
print('Model saved')

INFO:tensorflow:Froze 93 variables.
INFO:tensorflow:Converted 93 variables to const ops.
Model saved


In [10]:
print("Done!")

Done!
